In [1]:
import cantera as ct

In [2]:
from rmgpy import chemkin
from rmgpy import kinetics

full_path = "/work/westgroup/nora/Code/projects/halogens/refrigerants/singles/Burgess_Comments/cantera/difftool/"
inp_path = full_path + "pym_gas_copy.inp"
dict_path = full_path + "species_dictionary_2-BTP.txt"
modelN = chemkin.load_chemkin_file(inp_path, dict_path)

# full_pathR = "/work/westgroup/nora/Code/projects/halogens/refrigerants/singles/Burgess_Comments/cantera/difftool/"
# inp_pathR = full_pathR + "chem_annotated.inp"
# dict_pathR = full_pathR + "species_dictionary.txt"
# modelR = chemkin.load_chemkin_file(inp_pathR, dict_pathR)


full_pathR = "/work/westgroup/nora/Code/projects/halogens/refrigerants/singles/Burgess_Comments/cantera/difftool/"
inp_pathR = full_pathR + "no_halogens.inp"
dict_pathR = full_pathR + "no_halogens_dict.txt"
modelT = chemkin.load_chemkin_file(inp_pathR, dict_pathR)

In [10]:
import cantera as ct

#gasN = ct.Solution('/work/westgroup/nora/Code/projects/halogens/refrigerants/singles/Burgess_Comments/cantera/NIST/2-BTP_kinetics.cti')
gasR = ct.Solution('/work/westgroup/nora/Code/projects/halogens/refrigerants/singles/Burgess_Comments/cantera/RMG_David/chem_annotated.cti')

species_list = gasR.species()

species_list[11]

<Species Ar(10)>

In [11]:
import cantera as ct
from rmgpy import chemkin
import rmgpy.reaction
import rmgpy.species
import rmgpy.kinetics
import rmgpy.constants
import numpy as np
import matplotlib.pyplot as plt

In [14]:
rmg_thermo = rmgpy.thermo.nasa.NASAPolynomial()
rmg_thermo 

NASAPolynomial(coeffs=[0,0,0,0,0,0,0])

In [9]:
import cantera as ct 
from rmgpy import chemkin
from rmgpy import kinetics
import re

def compare_rxn_equations(modelR_reactions, modelN_reactions):
    '''
    Compares the reaction equations of each model. Uses to_cantera() on RMG model to eliminate chemkin_identifier 
    '''

################### mostly taken from diffmodel.py, compare_model_reactions() ###################

    #remove reactions with unknown species
    to_remove = []
    for reactionList in (modelR_reactions, modelN_reactions):
        for reaction in reactionList:
            for side in (reaction.products, reaction.reactants):
                for species in side:
                    if not species.molecule:
                        to_remove.append((reactionList, reaction))
                        print("Removing reaction {!r} that had unidentified species {!r}".format(reaction, species))
                        break
    for reactionList, reaction in to_remove:
        reactionList.remove(reaction)
    print(f'Following reactions removed because they have unknown species: {to_remove}')
    
    #find the common reactions and the unique reactions
    common_reactions = []
    common_reactions_equations = []
    unique_reactionsR = []
    unique_reactionsN = []
    copy_of_modelN_reactions = modelN_reactions.copy() #list 1
    copy_of_modelR_reactions = modelR_reactions.copy() #list 2

    
    for rxnN in copy_of_modelN_reactions:
        for rxnR in copy_of_modelR_reactions:  # make a copy so you don't remove from the list you are iterating over
            if rxnN.is_isomorphic(rxnR):
                common_reactions.append([rxnN, rxnR])
                common_reactions_equations.append([str(rxnN), str(rxnR)])
                # Remove reaction 2 from being chosen a second time.
                # Let each reaction only appear only once in the diff comparison.
                # Otherwise this miscounts number of reactions in model 2.
                copy_of_modelR_reactions.remove(rxnR)
                    
    for rxnN in copy_of_modelN_reactions:
        for rN, rR in common_reactions:
            if rxnN is rN:
                break
        else:
            unique_reactionsN.append(rxnN)
    for rxnR in copy_of_modelR_reactions:
        for rN, rR in common_reactions:
            if rxnR is rR:
                break
        else:
            unique_reactionsR.append(rxnR)
            
    different_kinetics_equations = []
    different_kinetics_reactions = []
            
    for rN, rR in common_reactions: 
        if (rN.kinetics.is_identical_to(rR.kinetics)==False) and (rN.kinetics.is_similar_to(rR.kinetics)==False):
            different_kinetics_equations.append(str(rR))
            different_kinetics_reactions.append(rR)
   # print([str(rxn) for rxn in unique_reactionsR])     
    return different_kinetics_equations, different_kinetics_reactions, common_reactions, common_reactions_equations, unique_reactionsR, unique_reactionsN 


def delete_halogenrxns_in_NIST(unique_reactionsN, unique_reactionsR):
    """
    Deletes reactions involving halogen in the NIST model. 
    """
    listN_no_halogens = []
    deleted_halogen_rxns = []

    for rxn_string in [str(rxn) for rxn in unique_reactionsN]: 
        matchF = re.search('F', rxn_string)
        matchBr = re.search('BR', rxn_string)
        if matchBr or matchF: 
            deleted_halogen_rxns.append(rxn_string)
        else: 
            listN_no_halogens.append(rxn_string)
    
    res = "\n".join("{}               {}".format(x, y) for x, y in zip([str(rxn) for rxn in unique_reactionsR], [str(rxn) for rxn in listN_no_halogens]))
    print(res)
    return


    
def execute(full_pathRMG, chemkinRMG, dictionaryRMG, full_pathNIST, chemkinNIST, dictionaryNIST): 
    '''
    loads in chemkin files and runs commands to execute the file
    '''
    full_pathN = full_pathNIST
    inp_pathN = full_pathNIST + chemkinNIST
    dict_pathN = full_pathNIST + dictionaryNIST
    modelN_species, modelN_reactions = chemkin.load_chemkin_file(inp_pathN, dict_pathN)

    full_pathR = full_pathRMG
    inp_pathR = full_pathRMG + chemkinRMG
    dict_pathR = full_pathRMG + dictionaryRMG
    modelR_species, modelR_reactions = chemkin.load_chemkin_file(inp_pathR, dict_pathR)
    
    different_kinetics_equations, different_kinetics_reactions, common_reactions, common_reactions_equations, unique_reactionsR, unique_reactionsN = compare_rxn_equations(modelR_reactions, modelN_reactions)
    delete_halogenrxns_in_NIST(unique_reactionsN, unique_reactionsR)
    return different_kinetics_equations, different_kinetics_reactions, common_reactions, common_reactions_equations, unique_reactionsR, unique_reactionsN
    

def main():   
    different_kinetics_equations, different_kinetics_reactions, common_reactions, common_reactions_equations, unique_reactionsR, unique_reactionsN = execute(
        "/work/westgroup/nora/Code/projects/halogens/refrigerants/singles/Burgess_Comments/cantera/difftool/",
        "no_halogens.inp",
        "no_halogens_dict.txt",
        "/work/westgroup/nora/Code/projects/halogens/refrigerants/singles/Burgess_Comments/cantera/difftool/",
        "pym_gas_copy.inp",
        "species_dictionary_2-BTP.txt"
        )

    
if __name__ == '__main__':
    main()


    

Following reactions removed because they have unknown species: []
Ar(10) + H2(8) <=> Ar(10) + H(5) + H(5)               H + H + H2 <=> H2 + H2
He(11) + H2(8) <=> He(11) + H(5) + H(5)               H + H + H2O <=> H2 + H2O
Ar(10) + O(6) + O(6) <=> Ar(10) + O2(4)               H + H + CO2 <=> H2 + CO2
He(11) + O(6) + O(6) <=> He(11) + O2(4)               HCO + H2O <=> CO + H + H2O
H2O(9) + H2O(9) <=> H(5) + OH(7) + H2O(9)               C + OH <=> CO + H
OH(7) + C(T)(17) <=> H(5) + CO(14)               C + O2 <=> CO + O
O2(4) + C(T)(17) <=> O(6) + CO(14)               CH + H <=> C + H2
H(5) + CH(18) <=> H2(8) + C(T)(17)               CH2 + O <=> H + HCO
H2(8) + CH(18) <=> CH3(20)               CH2 + O2 <=> HCO + OH
O2(4) + CH(18) <=> H(5) + CO2(15)               C + CH2 <=> C2H + H
O2(4) + CH(18) <=> OH(7) + CO(14)               CH2 + CH2 <=> C2H2 + H2
O2(4) + CH(18) <=> O(6) + H(5) + CO(14)               CH2* + O <=> CO + H2
O(6) + CH2(T)(19) <=> H(5) + H(5) + CO(14)               CH2* +

In [ ]:
#common reaction ['AR + CH2* <=> AR + CH2', 'Ar(10) + CH2(S)(26) <=> Ar(10) + CH2(T)(19)']

#unique reactions 

In [ ]:
def scramble_reactions_in_RMG_model(unique_reactionsN, unique_reactionsR):
    '''
    For situations like this: 
        RMG) 
    CH2* + O2 <=> CO + H + OH
    
    '''
    return 
    
    
def change_species_names_that_difftool_missed(unique_reactionsN, unique_reactionsR, common_reactions, common_reactions_equations, modelN_species, modelR_species):
    '''
    Of the unique reactions in the NIST model, 
    there are some species that are the same but are still coming up as different 
    (ex) CH4* (NIST) v. CH4(T)(19) (RMG)

    '''
    #makes a dictionary of common species in RMG and NIST
    common_species = {}
    for specN in modelN_species:
        for specT in modelR_species:
            if specN.smiles == specT.smiles:
                common_species[str(specN)] = str(specT)

    #manually fixing duplicates described below
    del common_species['CH2*']
    common_species['CH2*'] = 'CH2(T)(19)'

    #any duplicates in our relabeling?
    rev_dict = {}
    for key, value in common_species.items():
        rev_dict.setdefault(value, set()).add(key)  
    result = [key for key, values in rev_dict.items()
                              if len(values) > 1]
 
    if len(result)>0: 
        raise Exception('Duplicate in the common_species_label_dictionary that must be fixed.')
    #fix it manually above this line if this errors out  :(
    
    
    #among the unique reactions in NIST model, replace the NIST labels with the RMG labels in equation string. 
    #then split equation string across <=> to sort reactants and products for both NIST and RMG.
    #Then make dictionary of species in products and reactions for both equations and compare to see if numbers are same in both model equations. 
    #Should be exact equation if it is, and will be the ones diffmodel.py missed. 
    unique_reactionsN_with_RMG_labels = []
    for rxn in unique_reactionsN: 
        for reactant in rxn.reactants: 
            if reactant.to_chemkin() in common_species.keys(): 
                unique_reactionsN_with_RMG_labels.append(str(rxn).replace(reactant.to_chemkin(), common_species[reactant.to_chemkin()]))
   # print(unique_reactionsN_with_RMG_labels)
#     for rxn in unique_reactionsN: 
#         print(rxn.products, rxn.reactants) 




##################### now compare unique_reactionsN_with_RMG_labels to uniquereactionsRMG. might want to split them into reactants and products first
    return unique_reactionsN, unique_reactionsR, common_reactions, common_reactions_equation